### 선형회귀기반 분류 알고리즘 학습 - LogisticRegression + 교차검증

- 이진분류 및 다중분류 가능
- 분류지만 선형회귀식기반의 알고리즘이라서 Regresion

In [467]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

#### [1] 데이터 준비 및 확인

In [468]:
filename='../DATA/iris.csv'

In [469]:
irisDF= pd.read_csv(filename)

In [470]:
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


- 2개 품종만 추출 

In [471]:
irisDF.variety.unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [472]:
dataDF=irisDF[irisDF.variety != 'Virginica'].copy()
dataDF.shape

(100, 5)

- 꽃잎의 길이와 너비(petal.length, petal.width) 특성만 추출

In [473]:
dataDF.columns

Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width',
       'variety'],
      dtype='object')

In [474]:
dataDF=dataDF[dataDF.columns[2:]].copy()

In [475]:
dataDF.shape

(100, 3)

In [476]:
dataDF.head(3)

,petal.length,petal.width,variety
0,1.4,0.2,Setosa
1,1.4,0.2,Setosa
2,1.3,0.2,Setosa


In [477]:
pd.set_option('future.no_silent_downcasting', True)
labelToNum={'Setosa' : 0, 'Versicolor':1}
dataDF['variety_no']=dataDF.variety.replace(labelToNum )

In [478]:
dataDF.head(3)
dataDF.variety_no.unique()

array([0, 1], dtype=object)

In [479]:
dataDF.corr(numeric_only=True)

,petal.length,petal.width
petal.length,1.000000,0.979322
petal.width,0.979322,1.000000


In [480]:
## 시각화 그래프 

#### [2] 데이터 가공

- 피쳐/특성/독립과 타겟/클래스/종속변수 분리

In [481]:
featureDF = dataDF[dataDF.columns[:2]]
classDF = dataDF.variety_no
print(classDF)

0     0
1     0
2     0
3     0
4     0
     ..
95    1
96    1
97    1
98    1
99    1
Name: variety_no, Length: 100, dtype: object


In [482]:
print(f'featureDF : {featureDF.shape}  classDF : {classDF.shape}')

featureDF : (100, 2)  classDF : (100,)


- 훈련용/테스트용 분리

In [483]:
train_X, test_X, train_y, test_y = train_test_split(featureDF, classDF,
                                                    random_state=7,
                                                    test_size=0.2,
                                                    stratify=classDF)
test_X.shape
test_y.shape

(20,)

- 특성/피쳐 2개에 대한 스케일링 진행

In [484]:
from sklearn.preprocessing import StandardScaler

In [485]:
scaler = StandardScaler()

scaler.fit(train_X)

StandardScaler()

In [486]:
# 훈련데이터와 테스트 데이터 스케일링 진행
# ==> 훈련용 생성한 스케일러로 훈련용, 테스트용 변환 진행 
# ==> 향후 임의의 새로운 데이터 예측(predict)시에도 해당 스케일러 적용
train_scaled_X = scaler.transform(train_X)
test_scaled_X = scaler.transform(test_X)
test_scaled_X.shape

(20, 2)

#### [3] 학습 
- 메인학습 : 지도학습 - 분류
- 세부학습 : 선형모델 로지스틱회귀

In [487]:
# 학습모델 객체
logR=LogisticRegression(penalty='l1',solver='saga')
train_y=train_y.astype('int')
test_y=test_y.astype('int')
train_y

74    1
29    0
63    1
24    0
54    1
     ..
8     0
64    1
66    1
62    1
69    1
Name: variety_no, Length: 80, dtype: int64

In [488]:
# 학습 진행 ==> 훈련용 데이터 진행 fit(2D, 1D)
logR.fit(train_scaled_X, train_y)


c:\Users\gy931\anaconda3\envs\ML\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(penalty='l1', solver='saga')

In [489]:
# 학습 진행 후 설정되는 매개변수/파라미터 ===> 모델 파라미터 
# 파라메타명_
logR.classes_

array([0, 1])

In [490]:
logR.coef_

array([[3.4205357121, 1.2993134667]])

In [491]:
logR.intercept_

array([0.3705691931])

#### [4] 성능평가
- 이유 : 모델이 제대로 잘 만들어 졌는지 확인
- 과대적합/과소적합 체크
    * 과대적합/과적합/오버피팅(Overfitting) : 훈련 점수 > 테스트 점수
    * 과소적합/언더피팅(Underfitting) : 훈련과 테스트 점수 모두 낮음
    * 최적적합 : 훈련 점수와 테스트 점수 비슷 높은 점수 

- 점수 : 정확도(accuary_score)

In [492]:
train_score=logR.score(train_scaled_X, train_y)
test_scaled_X.dtype
test_y.dtype
test_score= logR.score(test_scaled_X, test_y)
print(test_scaled_X.shape)
print(test_y.shape)


(20, 2)
(20,)


In [493]:
print(f'train_score : {train_score}, test_score  : {test_score}')

train_score : 1.0, test_score  : 1.0


- 분류용 성능지표 ==>  sklearn.metrics 모듈

In [494]:
# 분류관련 성능지표 로딩
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report,confusion_matrix


In [495]:
# 테스트 데이터기반으로 예측값 추출
y_pred=logR.predict(test_scaled_X)
y_pred

array([1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0])

- [정밀도 점수] 

In [498]:
precision_score(test_y, y_pred)

1.0

In [499]:
recall_score(test_y, y_pred)

1.0

In [500]:
f1_score(test_y, y_pred)
confusion_matrix(test_y, y_pred)

array([[10,  0],
       [ 0, 10]])

In [501]:
print( classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



- 오차확인 : 손실/비용함수 => 알고리즘 마다 다름 log_loss()

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
np.set_printoptions(precision=10, suppress=True)

In [ ]:
# numpy에서의 DataFrame의 replace()와 동일 기능 변경 
test_y_ = np.where(test_y=='Versicolor', 0, 1)
y_pred_ = np.where(y_pred=='Versicolor', 0, 1)

loss = log_loss(test_y_, y_pred_)
print(f'loss : {loss :.6f}')

ValueError: y_true contains only one label (1). Please provide the true labels explicitly through the labels argument.

### 교차검증 ---------------------------------------------------------

- 데이터부족에 따른 과대적합/과적합/오버피팅을 해결하기 위한 방안
- 기본 데이터 분리 갯수 => 5 (지금껏 해당 기능을 사용해 보니 가장 성능이 좋았음)
- 데이터 구성
    * 입력 : 훈련데이터 , 타겟데이터
    * 분리 : 훈련데이터 ==> n등분(기:5)
        - 학습 완료 후 현재 모델에 대한 검사 진행 ==> 검증데이터필요
        - 1/n : 검증데이터로 사용
        - n-1/n : 훈련데이터로 사용
        - [중요] 검증데이터는 매번 변경
            * 검 훈 훈 훈 훈
            * 훈 검 훈 훈 훈
            * 훈 훈 검 훈 훈
            * 훈 훈 훈 검 훈
            * 훈 훈 훈 훈 검

In [ ]:
# 모듈 로딩 --------------------------------------------------
from sklearn.model_selection import cross_validate

In [ ]:
# 모델객체, 훈련데이터, 타겟데이터
result= cross_validate(logR, train_scaled_X, train_y)

In [ ]:
# 결과 result => dict 형태
for key, value in result.items(): print(key, value)

fit_time [0.004165411  0.0020551682 0.           0.           0.          ]
score_time [0.0009999275 0.0014493465 0.           0.           0.          ]
test_score [1. 1. 1. 1. 1.]


In [ ]:
# 모델객체, 훈련데이터, 타겟데이터 , 훈련결과, 최고의 모델 
result2= cross_validate(logR, train_scaled_X, train_y, 
                        return_estimator=True, 
                        return_train_score=True)

In [ ]:
for key, value in result2.items(): print(key, value)

fit_time [0.0032937527 0.0024936199 0.0027534962 0.           0.0055713654]
score_time [0.0009999275 0.0012173653 0.           0.           0.          ]
estimator [LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression()]
test_score [1. 1. 1. 1. 1.]
train_score [1. 1. 1. 1. 1.]


In [ ]:
# test_score와 train_score 값 비교해서 과대적합/과적합/오버피팅이 되지 않은
# 가장 좋음 모델 선택 후 추출
best_estimator=result2['estimator'][2]

In [ ]:
featureDF[:1]

,petal.length,petal.width
0,1.4,0.2


In [ ]:
new_data = pd.DataFrame([[1.4, 0.8]], columns=featureDF.columns)
new_data_ = scaler.transform(new_data)

In [ ]:
best_estimator.predict(new_data_)

array(['Setosa'], dtype=object)

In [ ]:
best_estimator.predict_proba(new_data_)

array([[0.8490584214, 0.1509415786]])